<img src="https://raw.githubusercontent.com/datagabe/pics/299e3b5241a8ab9fa8bb6289fc5f66217edf2b19/header.png?raw=true">
This notebook will walk you through the many ways that you can utilize SingleStoreDB today for your AI workloads.



Pre-Reqs:
* Spin up trial on SingleStore cloud at https://portal.singlestore.com
* Create a database named order_mgmt and load TPCH data. Use this link for instructions - https://docs.singlestore.com/cloud/load-data/load-a-type-of-data/example-load-tpc-h-data-into-singlestoredb-cloud/
* Create your own OPENAI_API_KEY from here - https://platform.openai.com/account/api-keys.

Resources:
* [SingleStore Vector Functions]()
* [Python LangChain](https://python.langchain.com/en/latest/getting_started/getting_started.html)

## Install Libraries (it will takes apprx. 20s)

In [46]:
!pip install langchain --quiet
!pip install openai --quiet
!pip install singlestoredb --quiet
!pip install tiktoken --quiet
!pip install unstructured --quiet
!pip install pdf2image --quiet
!pip install pdfminer.six --quiet
#to avoid cv2 error
!pip install opencv-python-headless --quiet

In [49]:
import ipywidgets as widgets
from IPython.display import display, HTML

import os
import getpass

from langchain import OpenAI, LLMChain
from langchain.sql_database import SQLDatabase
from langchain.agents.agent_toolkits import SQLDatabaseToolkit
from langchain.agents import create_sql_agent, AgentType, initialize_agent

# Define the variables
s2_host = connection_host
#s2_user = connection_user
#s2_pass = connection_password
s2_user = "admin"
s2_pass = ""
s2_port = connection_port

#os.environ["LANGCHAIN_TRACING"] = "false" # If you want to trace the execution of the program, set to "true"
os.environ["OPENAI_API_KEY"] = getpass.getpass("OpenAI API Key:")

# Create the interactive widgets
host_widget = widgets.Text(value=s2_host, description='S2 Host:', layout=widgets.Layout(width='auto'))
user_widget = widgets.Text(value=s2_user, description='S2 User:', layout=widgets.Layout(width='auto'))
pass_widget = widgets.Password(value=s2_pass, description='S2 Password:', layout=widgets.Layout(width='auto'))
port_widget = widgets.Text(value=s2_port, description='S2 Port:', layout=widgets.Layout(width='auto'))
url_widget = widgets.Text(description='S2 URL:', layout=widgets.Layout(width='350px'), disabled=True)

# Create helper fields
host_helper = widgets.HTML(value='<em>Enter the S2 host address.</em>', layout=widgets.Layout(color='black'))
user_helper = widgets.HTML(value='<em>Enter the S2 username.</em>', layout=widgets.Layout(color='black'))
pass_helper = widgets.HTML(value='<em>Enter the S2 password.</em>', layout=widgets.Layout(color='black'))
port_helper = widgets.HTML(value='<em>Enter the S2 port number.</em>', layout=widgets.Layout(color='black'))

# Create the update button
update_button = widgets.Button(description='Update', button_style='success')
update_button.style.button_color = '#AA00FF'

# Define the update function
def update_variables(_):
    global s2_host, s2_user, s2_pass, s2_port
    s2_host = host_widget.value
    s2_user = user_widget.value
    s2_pass = pass_widget.value
    s2_port = port_widget.value
    url_widget.value = f'mysql+pymysql://{s2_user}:{s2_pass}@{s2_host}:{s2_port}/'

# Attach the update function to the button's on-click event
update_button.on_click(update_variables)
update_variables('')

#display(HTML(f'<style>{style}</style>'))

# Display the widgets and the button in a vertical layout
widgets.VBox([
    widgets.HBox([host_widget, host_helper]),
    widgets.HBox([user_widget, user_helper]),
    widgets.HBox([pass_widget, pass_helper]),
    widgets.HBox([port_widget, port_helper]),
    widgets.HBox([url_widget]),
    widgets.HBox([update_button])
])


OpenAI API Key: ········


## Chatting over documents with SingleStoreDB

<img src="https://github.com/mnguyen1105/aws_dev_day_singapore_aug_2023/blob/main/vectorize.png?raw=true" width="1000">

### #1: Fetch PDF file (Note: Add leavcom.com to the firewall by selecting the Edit Firewall option in the top right)

In [54]:
from langchain.document_loaders import OnlinePDFLoader

loader = OnlinePDFLoader("http://leavcom.com/pdf/DBpdf.pdf")

data = loader.load()

In [16]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

print (f"You have {len(data)} document(s) in your data")
print (f"There are {len(data[0].page_content)} characters in your document")

You have 1 document(s) in your data
There are 13040 characters in your document


### #2: Chunking

In [17]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size = 2000, chunk_overlap = 0)
texts = text_splitter.split_documents(data)

print (f"You have {len(texts)} pages")

You have 8 pages


### #3: Embed as vector

In [19]:
from langchain.embeddings import OpenAIEmbeddings
#https://api.python.langchain.com/en/latest/embeddings/langchain.embeddings.openai.OpenAIEmbeddings.html
#text-embedding-ada-002
embedder = OpenAIEmbeddings()

In [20]:
%%sql

CREATE DATABASE IF NOT EXISTS pdf_db;
USE pdf_db;
DROP TABLE IF EXISTS pdf_docs2;

### #4: Store in table

In [21]:
from langchain.vectorstores import SingleStoreDB

os.environ["SINGLESTOREDB_URL"] =  f'{s2_user}:{s2_pass}@{s2_host}:{s2_port}/pdf_db'
docsearch = SingleStoreDB.from_documents(
    texts,
    embedder,
    table_name = "pdf_docs2"
)

In [23]:
%%sql
select count(1) from pdf_db.pdf_docs2 ;

count(1)
8


In [24]:
# Prompting SingleStoreDB for similarity with OpenAI embeddings
query_text = "Will object-oriented databases be commercially successful?"
docs = docsearch.similarity_search(query_text)
print(docs[0].page_content)

I N D U S T R Y T R E N D S

Whatever Happened to Object-Oriented Databases?

Neal Leavitt

are gaining in popularity and are ex- pected to outsell even relational data- bases by 2003. And OO databases (see the “OO Database Orientation” sidebar) are still minor players with solid but strictly niche markets. Sales of relational databases have grown considerably faster than the sales of OO databases, and annual worldwide RDBMS revenues are now about 50 times larger.

Rick Cattell, distinguished engineer at Sun Microsystems, indicated, “Object- oriented databases are doing just ﬁne, and the news of their demise is highly exag- gerated. While their market [share] isn’t as big, they continue to be used in areas like CAD (computer-aided design) and telecommunications, where RDBMSs are not well suited.”

A couple of years ago, industry

observers touted object-oriented databases as a technology on the rise, well suited for the emerg- ing Internet age.

However, said Michael Stonebraker, chief

### A step further with OpenAI

In [28]:
import openai

prompt = f"The user asked: {query_text}. The most similar text from the document is: {docs[0].page_content}"
response = openai.ChatCompletion.create(
    model="gpt-3.5-turbo",
    messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": prompt}
    ]
)

print(response['choices'][0]['message']['content'])

Based on the given information, it is suggested that object-oriented databases have not achieved the level of commercial success that was originally anticipated. While they continue to have a presence in niche markets such as CAD and telecommunications, relational databases have experienced significantly faster growth and larger revenues. Therefore, it can be inferred that object-oriented databases have not become commercially successful.


### or with LLAMA2 on AWS ( valid on 31st August, 2023 only)

In [31]:
import requests
import json

prompt = f"The user asked: {query_text}. The most similar text from the document is: {docs[0].page_content}"

url = "https://tihlngx6aa.execute-api.us-east-1.amazonaws.com/dev/llama22"
headers = {
  'Content-Type': 'application/json'
}
def generate_llama2_chat(prompt):
    dialogs = [
            [
                {
                    "role": "system",
                    "content": "You are a helpful assistant.",
                },
                {"role": "user", "content": prompt},
            ]
        ]

    payload = {
        "inputs":dialogs,
        "parameters": {"max_new_tokens": 512, "top_p": 0.9, "temperature": 0.6}
    }
    response = requests.request("POST", url, headers=headers, data=json.dumps(payload))
    return response
response = generate_llama2_chat(prompt)
print(json.loads(response.text)[0].get('generation').get('content'))

 Based on the information provided in the article, it appears that object-oriented databases have not achieved widespread commercial success. While they were once touted as a promising technology, their market share has not grown as expected, and they continue to occupy a small niche market.

According to the article, sales of relational databases have grown significantly faster than those of object-oriented databases, and annual worldwide RDBMS revenues are now about 50 times larger. This suggests that relational databases have been more successful in the marketplace.

The article also quotes Michael Stonebraker, a pioneer in the field of object-oriented databases, as saying that ORDBMSs will corner the market within five years. However, this has not come to pass, and object-oriented databases continue to be used in limited areas such as CAD and telecommunications, where relational databases are not well-suited.

Overall, it appears that while object-oriented databases have some uniqu

## Answering questions about your data in SingleStoreDB with OpenAI

### Connecting an OpenAI Agent to your SingleStore Data (Note: Follow instruction to create database named order_mgmt with data)

In [34]:
#db = SQLDatabase.from_uri(f"mysql+pymysql://admin:SingleStore1!@{s2_host}:3306/order_mgmt")
db = SQLDatabase.from_uri(f"mysql+pymysql://{s2_user}:{s2_pass}@{s2_host}:{s2_port}/order_mgmt")
llm = OpenAI(temperature=0, verbose=False)
toolkit = SQLDatabaseToolkit(db=db, llm=llm)
agent_executor = create_sql_agent(
    llm=OpenAI(temperature=0),
    toolkit=toolkit,
    verbose=True
)

/opt/conda/lib/python3.10/site-packages/langchain/utilities/sql_database.py:111: SAWarning: Unknown schema content: '  UNIQUE KEY `pk` (`l_orderkey`,`l_linenumber`) UNENFORCED RELY,'
  self._metadata.reflect(
/opt/conda/lib/python3.10/site-packages/langchain/utilities/sql_database.py:111: SAWarning: Unknown schema content: '  UNIQUE KEY `pk` (`s_suppkey`) UNENFORCED RELY,'
  self._metadata.reflect(
/opt/conda/lib/python3.10/site-packages/langchain/utilities/sql_database.py:111: SAWarning: Unknown schema content: '  UNIQUE KEY `pk` (`p_partkey`) UNENFORCED RELY,'
  self._metadata.reflect(
/opt/conda/lib/python3.10/site-packages/langchain/utilities/sql_database.py:111: SAWarning: Unknown schema content: '  UNIQUE KEY `pk` (`ps_partkey`,`ps_suppkey`) UNENFORCED RELY,'
  self._metadata.reflect(
/opt/conda/lib/python3.10/site-packages/langchain/utilities/sql_database.py:111: SAWarning: Unknown schema content: '  UNIQUE KEY `pk` (`c_custkey`) UNENFORCED RELY,'
  self._metadata.reflect(
/opt/

In [35]:
agent_executor.run("Write a SingleStore query that shows the highest paying customers per country, include how much they've spent, use the nation name")



> Entering new AgentExecutor chain...
Thought: I should use the sql_db_list_tables tool to get a list of tables in the database.
Action: sql_db_list_tables
Action Input: ""
Observation: customer, lineitem, nation, orders, part, partsupp, region, supplier
Thought: I should query the schema of the customer, orders, and nation tables.
Action: sql_db_schema
Action Input: "customer, orders, nation"
Observation: 
CREATE TABLE customer (
	c_custkey INTEGER(11) NOT NULL, 
	c_name VARCHAR(25) CHARACTER SET utf8 COLLATE utf8_general_ci NOT NULL, 
	c_address VARCHAR(40) CHARACTER SET utf8 COLLATE utf8_general_ci NOT NULL, 
	c_nationkey INTEGER(11) NOT NULL, 
	c_phone CHAR(15) CHARACTER SET utf8 COLLATE utf8_general_ci NOT NULL, 
	c_acctbal DECIMAL(15, 2) NOT NULL, 
	c_mktsegment CHAR(10) CHARACTER SET utf8 COLLATE utf8_general_ci NOT NULL, 
	c_comment VARCHAR(117) CHARACTER SET utf8 COLLATE utf8_general_ci NOT NULL
)

/*
3 rows from customer table:
c_custkey	c_name	c_address	c_nationkey	c_phone


Observation: [('Customer#011529199', Decimal('9999.99'), 'RUSSIA'), ('Customer#006344526', Decimal('9999.99'), 'ROMANIA'), ('Customer#003605586', Decimal('9999.99'), 'EGYPT'), ('Customer#010312182', Decimal('9999.99'), 'PERU'), ('Customer#001602401', Decimal('9999.99'), 'KENYA'), ('Customer#002134109', Decimal('9999.99'), 'MOROCCO'), ('Customer#006397913', Decimal('9999.99'), 'KENYA'), ('Customer#000508503', Decimal('9999.99'), 'UNITED STATES'), ('Customer#009953735', Decimal('9999.99'), 'ETHIOPIA'), ('Customer#014959900', Decimal('9999.99'), 'FRANCE')]
Thought:

 I now know the final answer
Final Answer: The highest paying customers per country are: Customer#011529199 in RUSSIA, Customer#006344526 in ROMANIA, Customer#003605586 in EGYPT, Customer#010312182 in PERU, Customer#001602401 in KENYA, Customer#002134109 in MOROCCO, Customer#006397913 in KENYA, Customer#000508503 in UNITED STATES, Customer#009953735 in ETHIOPIA, and Customer#014959900 in FRANCE. They have all spent $9999.99.

> Finished chain.


'The highest paying customers per country are: Customer#011529199 in RUSSIA, Customer#006344526 in ROMANIA, Customer#003605586 in EGYPT, Customer#010312182 in PERU, Customer#001602401 in KENYA, Customer#002134109 in MOROCCO, Customer#006397913 in KENYA, Customer#000508503 in UNITED STATES, Customer#009953735 in ETHIOPIA, and Customer#014959900 in FRANCE. They have all spent $9999.99.'

In [ ]:
%%sql
use order_mgmt;
SELECT c.c_name, c.c_acctbal, n.n_name FROM customer c INNER JOIN nation n ON c.c_nationkey = n.n_nationkey ORDER BY c.c_acctbal DESC LIMIT 10